In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
b = np.array([1,2,3,4,5,6,7,8,9,10])
c = np.linspace(0.1, 1, 10)
d = np.repeat(1, 10)

print(a/b, d/c)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] [10.          5.          3.33333333  2.5         2.          1.66666667
  1.42857143  1.25        1.11111111  1.        ]
